# SQL query from table names - Continued

In [ ]:
#from openai import OpenAI
#import os
#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv())

#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [1]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
import os

from google.colab import userdata

OPENAI_API_KEY = userdata.get('IronAPI')

## The old Prompt

In [6]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [9]:
context = [
    {
        'role': 'system',
        'content': """
you are a bot to assist in creating SQL commands, all your answers should start with "this is your SQL," and after that an SQL that can do what the user requests. Your Database is composed of a SQL database with some tables. Try to maintain the SQL order simple. Put the SQL command in white letters with a black background, and just after a simple and concise text explaining how it works. If the user asks for something that cannot be solved with an SQL command just answer something nice and simple, maximum 10 words, asking him for something that can be solved with SQL.
"""
    },
    {
        'role': 'system',
        'content': """
-- Define the structure and sample rows for the 'employees' table
CREATE TABLE employees (
  ID_usr int,
  name varchar
);

INSERT INTO employees (ID_usr, name) VALUES
(1, 'John Doe'),
(2, 'Jane Smith');

-- Define the structure and sample rows for the 'salary' table
CREATE TABLE salary (
  ID_usr int,
  year date,
  salary float
);

INSERT INTO salary (ID_usr, year, salary) VALUES
(1, '2023-01-01', 55000),
(2, '2023-01-01', 60000);

-- Define the structure and sample rows for the 'studies' table
CREATE TABLE studies (
  ID int,
  ID_usr int,
  educational_level int,
  Institution varchar,
  Years date,
  Speciality varchar
);

INSERT INTO studies (ID, ID_usr, educational_level, Institution, Years, Speciality) VALUES
(1, 1, 3, 'University A', '2018-01-01', 'Computer Science'),
(2, 2, 4, 'University B', '2017-01-01', 'Business Administration');

-- Few-Shot Samples with example queries and their corresponding SQL commands

-- Example Query 1: Select all employees
-- SQL Command:
SELECT * FROM employees;

-- Example Query 2: Find the salary of an employee by their ID
-- SQL Command:
SELECT salary FROM salary WHERE ID_usr = 1;

-- Example Query 3: List all studies for a specific user
-- SQL Command:
SELECT * FROM studies WHERE ID_usr = 2;
"""
    }
]


In [10]:
context.append( {'role':'system', 'content':"""
-- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

-- Example Query 1: Select all employees
-- SQL Command:
SELECT * FROM employees;

-- Example Query 2: Find the salary of an employee by their ID
-- SQL Command:
SELECT salary FROM salary WHERE ID_usr = 1;

-- Example Query 3: List all studies for a specific user
-- SQL Command:
SELECT * FROM studies WHERE ID_usr = 2;
"""
})

In [11]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [16]:
#new
context_user = context.copy()
print(return_CCRMSQL("Find the employee with the highest salary", context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query joins the 'employees' and 'salary' tables on the employee ID, then selects the employee name and salary, orders the result by salary in descending order, and limits the output to only the first row, which corresponds to the employee with the highest salary.


```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_Usr = s.ID_Usr
ORDER BY s.salary DESC
LIMIT 1;
```

In [17]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Find the employee with the highest salary", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query selects the name and salary of the employee with the highest salary by joining the "employees" and "salary" tables on the ID_usr column. It then orders the results by salary in descending order and limits the output to only the top result, which represents the employee with the highest salary.


This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name of the employee who is best paid by joining the "employees" table with the "salary" table on the ID_usr field. It then orders the result by salary in descending order and limits the output to only one row, which corresponds to the employee with the highest salary.

In [19]:
#new
print(return_CCRMSQL("Find the institution with the highest average salary among employees who have studied", context_user))

this is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the 'studies' and 'salary' tables on the user ID, calculates the average salary for each institution, orders the results by average salary in descending order, and then selects the institution with the highest average salary.


```sql
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_Usr = e.ID_Usr
JOIN salary sa ON e.ID_Usr = sa.ID_Usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

In [21]:
#old
print(return_CCRMSQL("Find the institution with the highest average salary among employees who have studied", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the three tables (employees, studies, and salary) based on the employee ID. It calculates the average salary for each institution where employees have studied, then orders the results by the average salary in descending order and limits the output to show only the institution with the highest average salary.


This is your SQL:
```sql
SELECT s.Institution
FROM studies s
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY AVG(sa.salary) DESC
LIMIT 1;
```

This SQL query joins the "studies" and "salary" tables on the ID_usr column. It then calculates the average salary for each institution, orders the results in descending order based on the average salary, and returns the institution with the highest average salary.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [22]:
print(return_CCRMSQL("find the highest salary among all employees", context_user))

this is your SQL:
```sql
SELECT MAX(salary) AS highest_salary FROM salary;
```
This SQL query selects the maximum salary from the `salary` table, giving you the highest salary among all employees.


In [23]:
print(return_CCRMSQL("find the highest salary among all employees", old_context_user))

This is your SQL:
```sql
SELECT MAX(salary) AS highest_salary
FROM salary;
```

This SQL query selects the maximum (highest) salary from the "salary" table, returning it as "highest_salary".


In [25]:
print(return_CCRMSQL("find the average salary of employees who studied grouped by universoty", context_user))

this is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution;
```

This SQL query joins the `studies` table with the `salary` table on the `ID_usr` column. It then calculates the average salary (`AVG`) for employees who studied at each university (`Institution`) by grouping the results based on the university.


In [26]:
print(return_CCRMSQL("find the average salary of employees who studied grouped by universoty", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS average_salary
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
GROUP BY s.Institution;
```

This SQL query joins the three tables (employees, studies, and salary) based on the employee ID. It then calculates the average salary of employees who studied at each university by grouping the results by the university's name.
